In [43]:
import cv2
from matplotlib import pyplot as plt
import mediapipe as mp
import numpy as np
import tensorflow as tf
import datetime
from moviepy.editor import VideoFileClip


In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # COLOR CONVERSION BGR 2 RGB
  image.flags.writeable = False                   # Image is no longer writeable
  results = model.process(image)                  # Make prediction
  image.flags.writeable = True                    # Image is now writeable
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR CONVERSION RGB 2 BGR
  return image, results

In [3]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw-left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw right hand connections

In [4]:
def draw_styled_landmarks (image, results):
    #Draw face connections
    mp_drawing.draw_landmarks(image, results. face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(88,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec (color=(80, 44, 121), thickness=2, circle_radius=2)
                             )
    #Draw Left hand connections
    mp_drawing.draw_landmarks (image, results. left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                               mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                               mp_drawing.DrawingSpec (color=(121,44,250), thickness=2, circle_radius=2) 
                              )
    #Draw right hand connections
    mp_drawing.draw_landmarks (image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                               mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                               mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                              )

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(21*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [18]:
model = tf.keras.models.load_model(r'C:\Users\ASUS\Desktop\project\mini\action.h5')

In [19]:
actions = np.array(['hello', 'thanks', 'iloveyou'])

In [44]:
# New detection variables
sentence = []
sequence = []  # Initialize sequence here
predictions = []
threshold = 0.7
val = ''
action_start_frame = None
video_url='https://firebasestorage.googleapis.com/v0/b/social-lips.appspot.com/o/posts%2Fvideo%2FWIN_20231216_17_31_50_Pro.mp4?alt=media&token=43da0e1a-b7b2-48fb-a4a5-b0edfb067142'
cap = cv2.VideoCapture(video_url)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
video_clip = VideoFileClip(video_url)

# Get the total duration in seconds
total_duration = video_clip.duration
print(total_duration)
actionrecord=""


# Create a dictionary to store action durations
action_durations = {action: [] for action in actions}

# ...

# Set mediopipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        if not ret:
            # Video ended, add end time to the last started action if needed
            if action_start_frame is not None:
                action_end_frame = cap.get(cv2.CAP_PROP_POS_MSEC)
                action_start_time = action_start_frame / 1000  # Convert milliseconds to seconds
              
                action_end_time =  total_duration
                action_durations[actionrecord].append((action_start_time, action_end_time))
            break  # End of video, break out of the loop

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw Landmarks
        draw_styled_landmarks(image, results)

        # Prediction Logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-29:]

        if len(sequence) == 29:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            val = actions[np.argmax(res)]
            predictions.append(np.argmax(res))

            # Check if the action has started
            if action_start_frame is None:
                if res[np.argmax(res)] > threshold:
                    action_start_frame = cap.get(cv2.CAP_PROP_POS_MSEC)
                    actionrecord=val
                    print("Action Start:", actionrecord, "at time", action_start_frame / 1000)  # Convert milliseconds to seconds

            # Check if the action has ended
            elif np.unique(predictions[-10:])[0] != np.argmax(res):
                if res[np.argmax(res)] <= threshold:
                    action_end_frame = cap.get(cv2.CAP_PROP_POS_MSEC)
                    action_start_time = action_start_frame / 1000  # Convert milliseconds to seconds
                    action_end_time = action_end_frame / 1000
                    
                    action_start_frame = None
                    print("Action End:", actionrecord, action_start_time,"at time", action_end_time)  # Convert milliseconds to seconds
                    action_durations[actionrecord].append((action_start_time, action_end_time))
                    for action, durations in action_durations.items():
    
                        print(f"{action} Durations:")
                        for start, end in durations:
                            if(end==0.00):
                                end=total_duration
                            print(f"{start:.2f}s - {end:.2f}s")

            # Viz Logic
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5:
                sentence = sentence[-5:]

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ''.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# ...





for action, durations in action_durations.items():
    
    print(f"{action} Durations:")
    for start, end in durations:
        if(end==0.00):
            end=total_duration
        print(f"{start:.2f}s - {end:.2f}s")
subtitle_string = "WEBVTT\n\n"

# Iterate over action durations
for action, durations in action_durations.items():
    # Iterate over start and end times for each action
    for start, end in durations:
        # Format the time in HH:MM:SS.sss
        start -= 29 / fps
       
        if(end==0.00):
            end=total_duration
        end -= 29 / fps
        start_time_str = datetime.timedelta(seconds=start)
        end_time_str = datetime.timedelta(seconds=end)
        
        # Append subtitle entry
        subtitle_string += f"{start_time_str} --> {end_time_str}\n{action}\n\n"

# Print the generated subtitle string
print(subtitle_string)
print(sentence)

13.1
1/1 [==============================] - 0s 28ms/step
Action Start: hello at time 3.7441666666666666
1/1 [==============================] - 0s 31ms/step
Action End: hello 3.7441666666666666 at time 10.192066666666667
hello Durations:
3.74s - 10.19s
thanks Durations:
iloveyou Durations:
1/1 [==============================] - 0s 13ms/step
Action Start: thanks at time 10.448133333333335
1/1 [==============================] - 0s 31ms/step
hello Durations:
3.74s - 10.19s
thanks Durations:
10.45s - 13.10s
iloveyou Durations:
WEBVTT

0:00:03.744167 --> 0:00:10.192067
hello

0:00:10.448133 --> 0:00:13.100000
thanks


['hello', 'thanks', 'hello']
